### Problem 1: Yield-Curves, forwards and swaps
All market instruments are constraints on a single arbitrage-free ZCB curve. Once we have ZCB prices we can construct forward rates, spot rates and swap rates. 
A zero-coupon bond price p(0,T) tells us: How much 1 unit of money paid at time T is worth today. 
Therefore the ZCB Curve is like the exchange rate between today and the future. 

1. **Spot rates**  
   What is the average discount from 0 to T. 6M EURIBOR fixing: rate of a loan from 0 to 0.5 years -> pins down the ZCB price p(0,0.5).

2. **Forward rates**  
  - Discount rate between two future dates
  - FRAs don’t give a single ZCB price → they link two ZCB points via a ratio


3. **Swap rates**  
  - Fixed rate that makes PV(fixed) = PV(floating)
  - Floating leg = sum of forward rates
  - Swap rate ≈ weighted average of forwards


#### 1.a: Fitting a yield curve of continuously compounded ZCB rates to the market data.
1. Start by writing up the given data in a structure as shown below.

In [ ]:
# DATA FOR YIELD CURVE CONSTRUCTION
Market_prices = np.array([x,x1,x2])
EURIBOR_fixing = [{"id": 0,"instrument": "libor","maturity": 1/2, "rate":0.03772}]
fra_market = [{"id": 1,"instrument": "fra","exercise": 1/12,"maturity": 7/12, "rate": 0.04026}]
## FRA = 1X7 means exercise = 1/12 and maturity = 7/12
swap_market = [{"id": 10,"instrument": "swap","maturity": 2, "rate": 0.05228, "float_freq": "semiannual", "fixed_freq": "annual","indices": []}]
## IRS = Interest rate swap IRS = 2Y -> Maturity 2 years. 
data_zcb = EURIBOR_fixing + fra_market + swap_market

2. Then choose which fit/type of interpolation you see most fit. 

In [ ]:
mesh = 1/12          # here is chosen a monthly mesh 
M = 360              # 30 years * 12
# interpolation options for the yield curve you can choose between linear, cubic spline and hermite.
interpolation_options = {"method": "hermite", "degree": 3, "transition": "smooth"} 
interpolation_options = {"method":"linear","transition": "smooth"}
interpolation_options = {"method":"nelson_siegel","transition": "smooth"}

# FITTING THE YIELD CURVE, T_fit produces knot points in years, R_fit the corresponding spot rates
T_fit, R_fit = fid.zcb_curve_fit(data_zcb, interpolation_options=interpolation_options)

# PLOTTING THE YIELD CURVE
T_inter = np.array([i*mesh for i in range(0,M+1)])
p_inter, R_inter, f_inter, T_inter = fid.zcb_curve_interpolate(T_inter,T_fit,R_fit,interpolation_options = interpolation_options)
#T_inter: Interpolated time points in years
#p_inter: Interpolated discount factors (zero-coupon prices)
#R_inter: Interpolated spot rates
#f_inter: Interpolated forward rates

3. Reporting the 6M, 1Y, 2Y, 5Y, 10Y, 15Y, 20Y and 30Y continuously compounded spot rates.
Given the I have fitted a continuous spot rate curve we computed above we now read off the spot rates at the maturities the exam asks for using the function below.

In [ ]:
# Function fid.for_values_in_list_find_value_return_value looks up for each maturity in the first list the corresponding spot rates from the interpolated curve
R_output = fid.for_values_in_list_find_value_return_value([0.5,1,2,5,10,15,20,30],T_inter,R_inter)
# r0 = short rate at time 0
r0 = R_inter[0]
print(f"Problem 1a - 6M,1Y,2Y,5Y,10Y,15Y,20Y,30Y spot rates from the fit: {np.round(R_output,5)}")

SSE: The SSE from the swap portion is of order $10^{-25}$ and from the FRA portion of order $10^{-10}$


#### 1.b: Discuss properties of spot and forward rates + curve type
- Spot rates should be continuous in maturity (similar cash flows should be discounted similarly).
- Forward rates should be well-behaved (preferably positive and smooth) to avoid unrealistic derivative prices.
- A smooth yield curve fit is chosen such that spot rates are continuous and forward rates are well-behaved. LOOK AT YOUR PLOT and see how the fitted curve fits these proporties and if it provides and arbitrage-free representation of the market term structure. 

#### 1.c: Computing 6M forward Euribor rates up to $T=10$ years and compute the 10Y par swap rate.
First we find discount factors at each semiannual date to compute the forward Euribor and the present values of the par swap. 

fid.forward_libor_rates_from_zcb_prices:
$$ L(0;T_{i-1},T_i)=\frac{p(0,T_{i-1}-p(0,T_i))}{\alpha p(0,T_i)}$$

fid.swap_rate_from_zcb_prices (par fixed rate at which swap value 0 at inception):
$$ R_{swap}=\frac{1-p(0,T_N)}{\sum^N_{i=1} \Delta_i p(0,T_i)}$$

Where $S=\sum \Delta_i p(0,T_i)$ is the accrual factor.



In [ ]:
#alpha_floating_leg means semiannual payments i.e. EURIBOR 6M
alpha_floating_leg = 0.5
#T_10Y_swap is the vector of payment dates for a 10Y swap with semiannual floating leg payments
#As we include period 0, we have 21 payments.
T_10Y_swap = np.array([i*alpha_floating_leg for i in range(0,21)])
#fid.function looks for each maturity in the first list the corresponding ZCB prices. 
#Finds: discount factors: p(0,0),p(0,0.5),p(0,1),...,p(0,10) needed to price cash flows on those dates.
p_10Y_swap = fid.for_values_in_list_find_value_return_value(T_10Y_swap,T_inter,p_inter)
#fid.forward function finds the forward Euribor for each 6M period. 
L_6M = fid.forward_libor_rates_from_zcb_prices(T_10Y_swap,p_10Y_swap,horizon = 1)
#Now we can compute the par swap rate for the 10Y swap
R_10Y_swap, S_10Y_swap = fid.swap_rate_from_zcb_prices(0,0,10,"annual",T_10Y_swap,p_10Y_swap)
print(f"Problem 1c - 10Y par swap rate: {R_10Y_swap}. The par swap rate is the fixed rate that exactly compensates for the floating leg in PV terms.")

The 10Y par swap rate is a weighted average of the forward rates corresponding to the floating leg. Meaning that the 10Y par swap rate is a weighted average of 6M forward Euribor rates.

### Problem 2: Pricing the interest rate cap

In [ ]:
#Defining the function
def fit_hwev_caplet_prices(param,price,strike_observed,T,p,scaling = 1):
    a, sigma = param
    caplet_price_fit = fid.caplet_prices_hwev(strike_observed,a,sigma,T,p)
    M = len(price)
    sse = 0
    for m in range(0,M):
        sse += scaling*(price[m] - caplet_price_fit[m])**2
    return sse

In [ ]:

# 2a) Fitting a Vasicek model to the yield curve
sigma_vasicek = 0.02
param_0 = 0.035, 6, 0.25
result = minimize(fid.fit_vasicek_sigma_fixed_obj,param_0,method = 'nelder-mead',args = (sigma_vasicek,R_inter,T_inter),options={'xatol': 1e-20,'disp': False})
r0_vasicek, a_vasicek, b_vasicek = result.x
print(f"Problem 2a - Vaseicek parameters: r0: {r0_vasicek}, a: {a_vasicek}, b: {b_vasicek}, sigma: {sigma_vasicek}, SSE: {result.fun}")
p_vasicek = fid.zcb_price_vasicek(r0_vasicek,a_vasicek,b_vasicek,sigma_vasicek,T_inter)
f_vasicek = fid.forward_rate_vasicek(r0_vasicek,a_vasicek,b_vasicek,sigma_vasicek,T_inter)
R_vasicek = fid.spot_rate_vasicek(r0_vasicek,a_vasicek,b_vasicek,sigma_vasicek,T_inter)


Problem 2a - Vaseicek parameters: r0: 0.024977946083671945, a: 5.574225555731182, b: 0.2885558485054699, sigma: 0.02, SSE: 0.0035706375190773863


In [ ]:
# 2b) Fitting the HWEV model to caplet prices
param_0 = 2.5, 0.018
result = minimize(fit_hwev_caplet_prices,param_0,method = 'nelder-mead',args = (price_caplet_market,strike_caplet_market,T_10Y_swap,p_10Y_swap),options={'xatol': 1e-20,'disp': False})
a_hwev, sigma_hwev = result.x
print(f"Problem 2b - HWEV parameters: a: {a_hwev}, sigma: {sigma_hwev}, SSE: {result.fun}")
caplet_price_fit = fid.caplet_prices_hwev(strike_caplet_market,a_hwev,sigma_hwev,T_10Y_swap,p_10Y_swap)
sigma_market, sigma_fit = np.nan*np.ones(N_caplet), np.nan*np.ones(N_caplet)
for i in range(2,N_caplet):
    sigma_market[i] = fid.black_caplet_iv(price_caplet_market[i],T_10Y_swap[i],strike_caplet_market,0.5,p_10Y_swap[i],L_6M[i],type_option = "call",prec = 1e-10)
    sigma_fit[i] = fid.black_caplet_iv(caplet_price_fit[i],T_10Y_swap[i],strike_caplet_market,0.5,p_10Y_swap[i],L_6M[i],type_option = "call",prec = 1e-10)


Problem 2b - HWEV parameters: a: 2.032982168566001, sigma: 0.020307157978552468, SSE: 5.033837247986103e-09


In [ ]:

# 2c) - Simulated trajectories of the Vasick and HWEV models
size_ci = 0.95
M_simul, T_simul = 500, 10
mesh_simul = T_simul/M_simul
t_simul = np.array([i*mesh_simul for i in range(0,M_simul+1)])
r_simul_vasicek = fid.simul_vasicek(r0_vasicek,a_vasicek,b_vasicek,sigma_vasicek,M_simul,T_simul,method = "euler")
mean_vasicek = fid.mean_vasicek(r0_vasicek, a_vasicek, b_vasicek, sigma_vasicek,t_simul)
lb_vasicek, ub_vasicek = fid.ci_vasicek(r0_vasicek, a_vasicek, b_vasicek, sigma_vasicek,t_simul,size_ci,type_ci = "two_sided")
lb_sd_vasicek, ub_sd_vasicek = fid.ci_vasicek(r0_vasicek, a_vasicek, b_vasicek, sigma_vasicek,np.inf,size_ci,type_ci = "two_sided")
print(f"Problem 2c - Vasicek model 2-sided CI under the stationary distribution: {lb_sd_vasicek}, {ub_sd_vasicek}")
f_simul, f_T_simul = fid.interpolate(t_simul,T_inter,f_inter,interpolation_options)
theta_hwev = fid.theta_hwev(t_simul,f_simul,f_T_simul,a_hwev,sigma_hwev)
r_simul_hwev = fid.simul_hwev(r0,t_simul,theta_hwev,a_hwev,sigma_hwev,method = "euler")
mean_hwev, var_hwev = fid.mean_var_hwev(a_hwev,sigma_hwev,t_simul,f_simul,f_T_simul)
lb_hwev, ub_hwev = fid.ci_hwev(a_hwev,sigma_hwev,t_simul,f_simul,f_T_simul,size_ci,type_ci = "two_sided")
print(f"HWEV model 2-sided CI under the stationary distribution: {lb_hwev[-1]}, {ub_hwev[-1]}")

Problem 2c - Vasicek model 2-sided CI under the stationary distribution: 0.040026014008215746, 0.06350616122240046
HWEV model 2-sided CI under the stationary distribution: 0.027563383164067812, 0.06704050223292485


In [ ]:

# 2d) - Price of the interest cap
strike_cap = 0.06
caplet_price_cap = fid.caplet_prices_hwev(strike_cap,a_hwev,sigma_hwev,T_10Y_swap,p_10Y_swap)
caplet_price_report = []
for i in [2,4,8,12,16,20]:
    caplet_price_report.append(np.round(10000*caplet_price_cap[i],4))
price_cap = sum(caplet_price_cap[2:])
premium_cap = alpha_floating_leg*price_cap/S_10Y_swap
print(f"Problem 2d - Caplet prices for T=1,2,4,6,8,10: {caplet_price_report}")
print(f"Problem 2d - price_cap: {10000*price_cap}, premium_cap: {10000*premium_cap}")


Problem 2d - Caplet prices for T=1,2,4,6,8,10: [np.float64(0.6687), np.float64(15.8546), np.float64(12.8065), np.float64(2.6277), np.float64(0.6122), np.float64(0.2263)]
Problem 2d - price_cap: 120.3749661196188, premium_cap: 8.033312945497595


# Problem 3 - Computing the price of the 3Y7Y payer swaption

In [ ]:
# 3a) 3Y7Y swaption price
T_n, T_N = 3, 10
beta = 0.55
R_swaption, S_swaption = fid.swap_rate_from_zcb_prices(0,T_n,T_N,"annual",T_10Y_swap,p_10Y_swap)
print(f"ATMF 3Y7Y par swap rate: {R_swaption}")
N_swaption = len(K_swaption_offset)
K_swaption = K_swaption_offset/10000 + R_swaption*np.ones(N_swaption)
price_swaption_market = np.zeros([N_swaption])
for i in range(0,N_swaption):
    price_swaption_market[i] = fid.black_swaption_price(iv_swaption_market[i],T_n,K_swaption[i],S_swaption,R_swaption)
print(f"Payer swaption with strike closest to K=0.06. strike: {K_swaption[7]}, price: {price_swaption_market[7]*10000} bps")


ATMF 3Y7Y par swap rate: 0.05503331345783611
Payer swaption with strike closest to K=0.06. strike: 0.06003331345783611, price: 41.195444387893204 bps


In [ ]:
# 3b) Fitting the SABR model
param_0 = 0.025, 0.48,-0.25
result = minimize(fid.fit_sabr_no_beta_obj,param_0,method = 'nelder-mead',args = (beta,iv_swaption_market,K_swaption,T_n,R_swaption),options={'xatol': 1e-8,'disp': False})
sigma_0, upsilon, rho = result.x
print(f"Parameters from the SABR fit where beta: {beta} are: sigma_0: {sigma_0}, upsilon: {upsilon}, rho: {rho}, SSE: {result.fun}")
iv_fit, price_fit = np.zeros([N_swaption]), np.zeros([N_swaption])
for i in range(0,N_swaption):
    iv_fit[i] = fid.sigma_sabr(K_swaption[i],T_n,R_swaption,sigma_0,beta,upsilon,rho)
    price_fit[i] = fid.black_swaption_price(iv_fit[i],T_n,K_swaption[i],S_swaption,R_swaption,type_option = "call")


Parameters from the SABR fit where beta: 0.55 are: sigma_0: 0.017875223851695068, upsilon: 0.5840153631324319, rho: -0.3439552092629677, SSE: 2.798846154816731e-05


In [ ]:
# 3c) Price and risk management of a strangle
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption,sigma_0,beta,upsilon,rho)
price_payer_swaption_init = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption,sigma_0,beta,upsilon,rho)
price_receiver_swaption_init = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption,type_option = "put")
print(f"Initially. Payer swaption: {price_payer_swaption_init*10000}, Receiver swaption: {price_receiver_swaption_init*10000}, Strangle: {price_payer_swaption_init*10000+price_receiver_swaption_init*10000}")
bps = 0.0001
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption + bps,sigma_0,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption + bps,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption + bps,sigma_0,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption + bps,type_option = "put")
print(f"Par swap rate UP 1 bps. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption - bps,sigma_0,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption - bps,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption - bps,sigma_0,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption - bps,type_option = "put")
print(f"Par swap rate DOWN 1 bps. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption,sigma_0 + 0.001,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption,sigma_0 + 0.001,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption,type_option = "put")
print(f"Volatility 0.001 UP. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")
iv_payer_swaption = fid.sigma_sabr(K_swaption[8],T_n,R_swaption,sigma_0 - 0.001,beta,upsilon,rho)
price_payer_swaption = fid.black_swaption_price(iv_payer_swaption,T_n,K_swaption[8],S_swaption,R_swaption,type_option = "call")
iv_receiver_swaption = fid.sigma_sabr(K_swaption[4],T_n,R_swaption,sigma_0 - 0.001,beta,upsilon,rho)
price_receiver_swaption = fid.black_swaption_price(iv_receiver_swaption,T_n,K_swaption[4],S_swaption,R_swaption,type_option = "put")
print(f"Volatility 0.001 DOWN. Payer swaption: {price_payer_swaption*10000}, Receiver swaption: {price_receiver_swaption*10000}, Strangle: {price_payer_swaption*10000+price_receiver_swaption*10000}, Difference: {10000*(price_payer_swaption+price_receiver_swaption-price_payer_swaption_init-price_receiver_swaption_init)}")


Initially. Payer swaption: 16.053096667407942, Receiver swaption: 31.424116085775022, Strangle: 47.47721275318297
Par swap rate UP 1 bps. Payer swaption: 16.35234104159374, Receiver swaption: 31.13234003219374, Strangle: 47.48468107378748, Difference: 0.007468320604511747
Par swap rate DOWN 1 bps. Payer swaption: 15.76079054120206, Receiver swaption: 31.72019106310972, Strangle: 47.48098160431178, Difference: 0.003768851128812742
Volatility 0.001 UP. Payer swaption: 18.57246035677486, Receiver swaption: 35.03239454085507, Strangle: 53.604854897629934, Difference: 6.127642144446972
Volatility 0.001 DOWN. Payer swaption: 13.72821984857286, Receiver swaption: 27.959714097606827, Strangle: 41.68793394617968, Difference: -5.789278807003277


# Problem 5 - Short rate dynamics

In [ ]:
# problem 5
def mean_var(x0,a,b,sigma,y0,gamma,phi,T):
    N = len(T)
    mean = np.zeros(N)
    var = np.zeros(N)
    for n, t in enumerate(T):
        mean[n] = x0*np.exp(-gamma*T[n]) + y0*np.exp(-a*T[n]) + b/a*(1-np.exp(-a*T[n]))
        var[n] = phi**2/(2*gamma)*(1-np.exp(-2*gamma*T[n])) + sigma**2/(2*a)*(1-np.exp(-2*a*T[n]))
    return mean, var

# 5c) Simulating the factors driving the short rate and the short rate itself
x0, gamma, phi = 0, 32, 0.03
y0, a, b, sigma = 0.03, 0.5, 0.025, 0.015
M = 1000
T_st = 1
T_lt = 10
size_ci = 0.95
z = norm.ppf(size_ci + 0.5*(1-size_ci),0,1)
delta_st = T_st/M
t_simul_st = np.array([i*delta_st for i in range(0,M+1)])
X_st = fid.simul_vasicek(x0,gamma,0,phi,M,T_st)
Y_st = fid.simul_vasicek(y0,a,b,sigma,M,T_st)
r_st = X_st + Y_st
mean_st, var_st = mean_var(x0,a,b,sigma,y0,gamma,phi,t_simul_st)
lb_st, ub_st = mean_st - z*np.sqrt(var_st), mean_st + z*np.sqrt(var_st)
delta_lt = T_lt/M
t_simul_lt = np.array([i*delta_lt for i in range(0,M+1)])
X_lt = fid.simul_vasicek(x0,gamma,0,phi,M,T_lt)
Y_lt = fid.simul_vasicek(y0,a,b,sigma,M,T_lt)
r_lt = X_lt + Y_lt
mean_lt, var_lt = mean_var(x0,a,b,sigma,y0,gamma,phi,t_simul_lt)
lb_lt, ub_lt = mean_lt - z*np.sqrt(var_lt), mean_lt + z*np.sqrt(var_lt)
